My aim  
Marks (Maths, Python, SQL) matter MOST → Communication → Mini Projects → Placement Readiness → Attendance (least)

In [1]:

!pip install lightgbm -q


In [30]:




import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    accuracy_score,
    confusion_matrix
)

import warnings
warnings.filterwarnings("ignore")




In [31]:


DATA_PATH = "/content/sample_data/student_placement_predictor_dataset.csv"  # change path if needed
df = pd.read_csv(DATA_PATH)

print("Initial shape:", df.shape)



Initial shape: (1000, 10)


In [32]:

# Drop useless columns
df.drop(columns=['Student_ID', 'Name'], inplace=True)

# Encode target
df['Placed'] = df['Placed'].map({'Yes': 1, 'No': 0})



In [33]:


df['Maths_w']   = df['Maths'] * 2.0
df['Python_w']  = df['Python'] * 2.0
df['SQL_w']     = df['SQL'] * 2.0

df['Comm_w']    = df['Communication_Score'] * 1.5

# log1p for skewed count feature
df['Mini_w']    = np.log1p(df['Mini_Projects']) * 1.0

df['Ready_w']   = df['Placement_Readiness_Score'] * 0.7
df['Attend_w']  = df['Attendance'] * 0.5


In [34]:


features = [
    'Maths_w', 'Python_w', 'SQL_w',
    'Comm_w', 'Mini_w',
    'Ready_w', 'Attend_w'
]

X = df[features]
y = df['Placed']


In [35]:


X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train shape:", X_train.shape)
print("Test shape :", X_test.shape)



Train shape: (800, 7)
Test shape : (200, 7)


In [36]:

scale_pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()
print("Scale_pos_weight:", scale_pos_weight)


Scale_pos_weight: 0.3559322033898305


In [37]:


model = lgb.LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42
)

model.fit(X_train, y_train)



[LightGBM] [Info] Number of positive: 590, number of negative: 210
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000130 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 578
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.737500 -> initscore=1.033015
[LightGBM] [Info] Start training from score 1.033015
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.05, n_estimators=500,
               random_state=42, scale_pos_weight=np.float64(0.3559322033898305),
               subsample=0.8)

In [38]:

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC :", roc_auc_score(y_test, y_prob))

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))




Accuracy: 0.995
ROC-AUC : 1.0

Classification Report:

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        53
           1       1.00      0.99      1.00       147

    accuracy                           0.99       200
   macro avg       0.99      1.00      0.99       200
weighted avg       1.00      0.99      1.00       200

Confusion Matrix:

[[ 53   0]
 [  1 146]]


In [39]:

importance = pd.Series(
    model.feature_importances_,
    index=features
).sort_values(ascending=False)

print("\nFeature Importance:\n")
print(importance)




Feature Importance:

Ready_w     1910
SQL_w        938
Python_w     776
Comm_w       717
Maths_w      595
Attend_w     525
Mini_w       523
dtype: int32


In [40]:
custom_threshold = 0.65
y_custom = (y_prob > custom_threshold).astype(int)

print("\nCustom Threshold Accuracy:",
      accuracy_score(y_test, y_custom))


Custom Threshold Accuracy: 0.995


In [29]:
import joblib

joblib.dump(model, "placement_model.pkl")
print("Model saved successfully!")


Model saved successfully!


In [42]:


!pip install lightgbm gradio -q

import pandas as pd
import numpy as np
import lightgbm as lgb
import gradio as gr

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")



DATA_PATH = "/content/sample_data/student_placement_predictor_dataset.csv"   # change path if needed
df = pd.read_csv(DATA_PATH)

# Cleaning
df.drop(columns=['Student_ID', 'Name'], inplace=True)
df['Placed'] = df['Placed'].map({'Yes': 1, 'No': 0})

# Feature engineering
df['Maths_w']   = df['Maths'] * 2
df['Python_w']  = df['Python'] * 2
df['SQL_w']     = df['SQL'] * 2
df['Comm_w']    = df['Communication_Score'] * 1.5
df['Mini_w']    = np.log1p(df['Mini_Projects'])
df['Ready_w']   = df['Placement_Readiness_Score'] * 0.7
df['Attend_w']  = df['Attendance'] * 0.5

features = [
    'Maths_w','Python_w','SQL_w',
    'Comm_w','Mini_w','Ready_w','Attend_w'
]

X = df[features]
y = df['Placed']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Handle imbalance
scale_pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()



model = lgb.LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42
)

model.fit(X_train, y_train)

print("Model ROC-AUC:",
      roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))


def predict_placement(maths, python, sql, comm, mini, readiness, attendance):

    # Apply SAME transformations
    data = np.array([[
        maths * 2,
        python * 2,
        sql * 2,
        comm * 1.5,
        np.log1p(mini),
        readiness * 0.7,
        attendance * 0.5
    ]])

    prob = model.predict_proba(data)[0][1]
    decision = "PLACED " if prob >= 0.65 else "NOT PLACED ❌"

    return decision, round(prob, 3)


app = gr.Interface(
    fn=predict_placement,
    inputs=[
        gr.Slider(0,100, label="Maths Marks"),
        gr.Slider(0,100, label="Python Marks"),
        gr.Slider(0,100, label="SQL Marks"),
        gr.Slider(0,100, label="Communication Score"),
        gr.Slider(0,10,  step=1, label="Mini Projects"),
        gr.Slider(0,100, label="Placement Readiness"),
        gr.Slider(0,100, label="Attendance %"),
    ],
    outputs=[
        gr.Textbox(label="Placement Decision"),
        gr.Number(label="Placement Probability")
    ],
    title="🎓 Student Placement Predictor",
    description="LightGBM model prioritizing technical skills over attendance & soft skills"
)

app.launch()

[LightGBM] [Info] Number of positive: 590, number of negative: 210
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000139 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 578
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.737500 -> initscore=1.033015
[LightGBM] [Info] Start training from score 1.033015
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga